In [1]:
import pandas as pd 
from nltk.corpus import stopwords
from sklearn import neural_network
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import SGDClassifier
from sklearn.externals import joblib
import pickle

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df = pd.read_csv('preprocessed_rest.csv')
df  = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8539 entries, 0 to 8538
Data columns (total 2 columns):
polarity    8539 non-null object
text        8539 non-null object
dtypes: object(2)
memory usage: 200.1+ KB


In [3]:
df['polarity'].value_counts()

p    6863
n    1676
Name: polarity, dtype: int64

In [4]:
pipeline = Pipeline([
        ('bow', CountVectorizer(ngram_range=(1,2))),  # strings to token integer counts
        ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
        #('Normalize', preprocessing.Normalizer()),
        #('reduce_dim',TruncatedSVD(n_components = 150)),
        #('c',neural_network.MLPClassifier( hidden_layer_sizes=(100,100), alpha = 0.01,learning_rate_init=0.01, max_iter=50, )),
        ('c',LinearSVC()),
    ])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['polarity'], test_size=0.33, random_state=42)
tdf = pd.DataFrame({'polarity': y_train, 'text': X_train })
for i in range(0,100,5):
    bdf = tdf.drop(tdf.query('polarity =="p" ').sample(frac=i/100).index)
    clf = pipeline.fit(bdf['text'],bdf['polarity'])
    pred=pipeline.predict(X_test)
    print(confusion_matrix(y_test,pred))
    print(classification_report(y_test,pred))
    print(" i , Accuracy : ", i , accuracy_score(pred,y_test))

[[ 349  217]
 [  35 2217]]
             precision    recall  f1-score   support

          n       0.91      0.62      0.73       566
          p       0.91      0.98      0.95      2252

avg / total       0.91      0.91      0.90      2818

 i , Accuracy :  0 0.9105748757984387
[[ 354  212]
 [  42 2210]]
             precision    recall  f1-score   support

          n       0.89      0.63      0.74       566
          p       0.91      0.98      0.95      2252

avg / total       0.91      0.91      0.90      2818

 i , Accuracy :  5 0.9098651525904897
[[ 367  199]
 [  41 2211]]
             precision    recall  f1-score   support

          n       0.90      0.65      0.75       566
          p       0.92      0.98      0.95      2252

avg / total       0.91      0.91      0.91      2818

 i , Accuracy :  10 0.914833215046132
[[ 373  193]
 [  46 2206]]
             precision    recall  f1-score   support

          n       0.89      0.66      0.76       566
          p       0.92    

In [10]:
#best balanced result -> frac = .50
bdf = tdf.drop(tdf.query('polarity =="p" ').sample(frac = .50).index)
clf = pipeline.fit(bdf['text'],bdf['polarity'])
pred=pipeline.predict(X_test)
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("Accuracy : ", accuracy_score(pred,y_test))

[[ 426  140]
 [ 104 2148]]
             precision    recall  f1-score   support

          n       0.80      0.75      0.78       566
          p       0.94      0.95      0.95      2252

avg / total       0.91      0.91      0.91      2818

Accuracy :  0.9134137686302342


In [11]:
joblib.dump(clf, 'restaurant.joblib') 

['restaurant.joblib']

In [12]:
filename = 'restaurant.pickle'
pickle.dump(clf, open(filename, 'wb'))